In [1]:
!pip install jieba
!git clone https://github.com/ljyslyc/comment-analysis.git

Cloning into 'comment-analysis'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 127 (delta 20), reused 25 (delta 9), pack-reused 83
Receiving objects: 100% (127/127), 82.49 MiB | 38.59 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [3]:
import pandas as pd
from pathlib import Path
import numpy as np

mypath = Path("/content/comment-analysis/")
Comment_datas = pd.read_csv(mypath/'ml-bayes-demo/Comment_datas.csv')
print(Comment_datas.head())
print(Comment_datas.shape)

                                             Comment  IsGreet
0                               个人吃加了个羊腿不错的一家西餐厅还会光顾        1
1  我和男朋友是秀玉的老顾客了第一次认识就是在秀玉吃的饭他们家的环境优雅菜品味道也挺不错今天和男...        1
2  每次来秀玉觉得心情很好因为环境不错东西又很好吃因为没有拍照的习惯只能拿朋友做的东西来凑了貌似...        1
3  和室友一起来的这里位置很好找整个装饰也很有格调适合人们休息点了香辣咖喱鸡焗饭冰淇淋水果圣代奶...        1
4                                 还不错吧环境优雅菜品种类多味道还可以        1
(19971, 2)


In [7]:
#加在停用词库
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file,encoding='gbk') as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list
stop_words_file = mypath/"stopwords/中文停用词库.txt"
stopwords = get_custom_stopwords(stop_words_file)
print(stopwords)

['$', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', '_', '“', '”', '、', '。', '《', '》', '一', '一些', '一何', '一切', '一则', '一方面', '一旦', '一来', '一样', '一般', '一转眼', '万一', '上', '上下', '下', '不', '不仅', '不但', '不光', '不单', '若果', '不只', '不外乎', '不如', '不妨', '不尽', '不尽然', '不得', '不怕', '不惟', '不成', '不拘', '不料', '不是', '不比', '不然', '不特', '不独', '不管', '不至于', '不若', '不论', '不过', '不问', '与', '与其', '与其说', '与否', '与此同时', '且', '且不说', '且说', '两者', '个', '个别', '临', '为', '为了', '为什么', '为何', '为止', '为此', '为着', '乃', '乃至', '乃至于', '么', '之', '之一', '之所以', '之类', '乌乎', '乎', '乘', '也', '也好', '也罢', '了', '二来', '于', '于是', '于是乎', '云云', '云尔', '些', '亦', '人', '人们', '人家', '什么', '什么样', '今', '介于', '仍', '仍旧', '从', '从此', '从而', '他', '他人', '他们', '以', '以上', '以为', '以便', '以免', '以及', '以故', '以期', '以来', '以至', '以至于', '以致', '们', '任', '任何', '任凭', '似的', '但', '但凡', '但是', '何', '何以', '何况', '何处', '何时', '余外', '作为', '你', '你们', '使', '使得', '例如', '依', '依据', '依照', '便于', '俺', '俺们', '倘', '倘使', '倘或', '倘然', '倘若', '借', '假使', '假如', '假若', '傥然', '像', '儿', '先不先', '光是', '全体', '

In [8]:
X = Comment_datas[['Comment']]
X.head()

,Comment
0,个人吃加了个羊腿不错的一家西餐厅还会光顾
1,我和男朋友是秀玉的老顾客了第一次认识就是在秀玉吃的饭他们家的环境优雅菜品味道也挺不错今天和男...
2,每次来秀玉觉得心情很好因为环境不错东西又很好吃因为没有拍照的习惯只能拿朋友做的东西来凑了貌似...
3,和室友一起来的这里位置很好找整个装饰也很有格调适合人们休息点了香辣咖喱鸡焗饭冰淇淋水果圣代奶...
4,还不错吧环境优雅菜品种类多味道还可以


In [9]:
Y = Comment_datas.IsGreet
Y.head()

0    1
1    1
2    1
3    1
4    1
Name: IsGreet, dtype: int64

In [10]:
import jieba
import numpy as np
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

X['cutted_comment'] = X.Comment.apply(chinese_word_cut)
X.head()


Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.971 seconds.
Prefix dict has been built succesfully.


,Comment,cutted_comment
0,个人吃加了个羊腿不错的一家西餐厅还会光顾,个人 吃加 了 个 羊腿 不错 的 一家 西餐厅 还会 光顾
1,我和男朋友是秀玉的老顾客了第一次认识就是在秀玉吃的饭他们家的环境优雅菜品味道也挺不错今天和男...,我 和 男朋友 是秀玉 的 老 顾客 了 第一次 认识 就是 在秀玉 吃 的 饭 他们 家 ...
2,每次来秀玉觉得心情很好因为环境不错东西又很好吃因为没有拍照的习惯只能拿朋友做的东西来凑了貌似...,每次 来秀玉 觉得 心情 很 好 因为 环境 不错 东西 又 很 好吃 因为 没有 拍照 的...
3,和室友一起来的这里位置很好找整个装饰也很有格调适合人们休息点了香辣咖喱鸡焗饭冰淇淋水果圣代奶...,和 室友 一 起来 的 这里 位置 很 好找 整个 装饰 也 很 有 格调 适合 人们 休息...
4,还不错吧环境优雅菜品种类多味道还可以,还 不错 吧 环境 优雅 菜品 种类 多 味道 还 可以


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=1)
vect = CountVectorizer()
term_matrix = pd.DataFrame(vect.fit_transform(X_train.cutted_comment).toarray(), columns=vect.get_feature_names())
term_matrix.head()

,一一,一万个,一万次,一下,一下下,一下子,一不小心,一丝,一两个,一两回,一个,一个三岁,一个两岁,一个个,一个五岁,一个劲,一个劲儿,一个千,一个半,一个多,一个月,一个样,一个点,一个舒服,一之太,一九九五年,一些,一人,一人点,一人花,一件,一份,一伙,一会,一会儿,一位,一体,一共,一出,一分,...,麻烦,麻花,麻辣,麻麻,黄伟君,黄冈,黄桃,黄桃配,黄瓜,黄石,黄维君,黄色,黄花菜,黄豆,黄道吉日,黄金,黄鳝,黎咯,黑乎乎,黑名单,黑暗,黑椒,黑浇汁,黑米,黑糊糊,黑糖,黑胡椒,黑色,黑衣服,默认,默默,默默地,點貴,鼓鼓的,鼻孔,鼻梁骨,齐全,齐差,齐醉,龟速
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
max_df = 0.8 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
min_df = 3 # 在低于这一数量的文档中出现的关键词（过于独特），去除掉。
vect = CountVectorizer(max_df = max_df, 
                       min_df = min_df, 
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b', 
                       stop_words=frozenset(stopwords))
term_matrix = pd.DataFrame(vect.fit_transform(X_train.cutted_comment).toarray(), columns=vect.get_feature_names())
term_matrix.head()

,一下,一下下,一不小心,一个,一个半,一个多,一个月,一人,一人点,一份,一会,一会儿,一位,一共,一分,一分货,一分钱,一到,一副,一半,一去,一口,一句,一只,一吃,一向,一周,一圈,一块,一堆,一声,一壶,一大,一大块,一大堆,一天,一如,一如既往,一定,一家,...,马上,马马虎虎,验证,验证码,骨头,骨头汤,高兴,高大,高峰,高峰期,高性价比,高端,高脚杯,鱼翅,鱿鱼,鲜嫩,鲜美,鲜虾,鳕鱼,鳗鱼,鳝鱼,鸡中,鸡中节,鸡排,鸡翅,鸡肉,鸡蛋,麻烦,黄桃,黄瓜,黄石,黄维君,黄豆,黄金,黑椒,黑米,黑胡椒,黑衣服,默默,齐全
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score


nb = MultinomialNB()
pipe = make_pipeline(vect, nb)
pipe.steps

[('countvectorizer',
  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                  dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                  lowercase=True, max_df=0.8, max_features=None, min_df=3,
                  ngram_range=(1, 1), preprocessor=None,
                  stop_words=frozenset({'', '$', '0', '1', '2', '3', '4', '5',
                                        '6', '7', '8', '9', '?', '_', '“', '”',
                                        '、', '。', '《', '》', '一', '一些', '一何', '一切',
                                        '一则', '一方面', '一旦', '一来', '一样', '一般', ...}),
                  strip_accents=None, token_pattern='(?u)\\b[^\\d\\W]\\w+\\b',
                  tokenizer=None, vocabulary=None)),
 ('multinomialnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]

In [14]:
cross_val_score(pipe, X_train.cutted_comment, y_train, cv=5, scoring='accuracy').mean()

0.9540654520457827

In [15]:
pipe.fit(X_train.cutted_comment, y_train)
pipe.predict(X_test.cutted_comment)

array([1, 1, 1, ..., 1, 1, 1])

In [16]:
from sklearn import metrics
y_pred = pipe.predict(X_test.cutted_comment)
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))

0.9541357901061486
[[  37  121]
 [ 108 4727]]
